In [1]:
%matplotlib inline

import math
import datetime
import matplotlib.pyplot as plt
import operator
import random
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas_summary import DataFrameSummary
from isoweek import Week
from tqdm import tqdm
pd.options.display.max_columns = 50
from sklearn.externals import joblib

In [44]:
from keras.models import load_model
from keras.layers import Input, Dense, BatchNormalization, Dropout, Activation
from keras.models import Model

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error

from xgboost import XGBRegressor

In [2]:
df_test = pd.read_csv('test.csv')
df_train = pd.read_csv('train.csv')
df_stores = pd.read_csv('store.csv')
df_train = df_train.merge(df_stores, on='Store')
df_test = df_test.merge(df_stores, on='Store')

/home/adityasidharta/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_train.columns

Index([u'Store', u'DayOfWeek', u'Date', u'Sales', u'Customers', u'Open',
       u'Promo', u'StateHoliday', u'SchoolHoliday', u'StoreType',
       u'Assortment', u'CompetitionDistance', u'CompetitionOpenSinceMonth',
       u'CompetitionOpenSinceYear', u'Promo2', u'Promo2SinceWeek',
       u'Promo2SinceYear', u'PromoInterval'],
      dtype='object')

In [4]:
df_test.columns

Index([u'Id', u'Store', u'DayOfWeek', u'Date', u'Open', u'Promo',
       u'StateHoliday', u'SchoolHoliday', u'StoreType', u'Assortment',
       u'CompetitionDistance', u'CompetitionOpenSinceMonth',
       u'CompetitionOpenSinceYear', u'Promo2', u'Promo2SinceWeek',
       u'Promo2SinceYear', u'PromoInterval'],
      dtype='object')

In [5]:
df_train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [6]:
df_test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,1,4,2015-09-17,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,857,1,3,2015-09-16,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1713,1,2,2015-09-15,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,2569,1,1,2015-09-14,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,3425,1,7,2015-09-13,0.0,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [7]:
DataFrameSummary(df_train).summary()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
count,1.01721e+06,1.01721e+06,NaN,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,NaN,1.01721e+06,NaN,NaN,1.01457e+06,693861,693861,1.01721e+06,509178,509178,NaN
mean,558.43,3.99834,NaN,5773.82,633.146,0.830107,0.381515,NaN,0.178647,NaN,NaN,5430.09,7.22287,2008.69,0.500564,23.2691,2011.75,NaN
std,321.909,1.99739,NaN,3849.93,464.412,0.375539,0.485759,NaN,0.383056,NaN,NaN,7715.32,3.21183,5.99264,0.5,14.096,1.66287,NaN
min,1,1,NaN,0,0,0,0,NaN,0,NaN,NaN,20,1,1900,0,1,2009,NaN
25%,280,2,NaN,3727,405,1,0,NaN,0,NaN,NaN,710,4,2006,0,13,2011,NaN
50%,558,4,NaN,5744,609,1,0,NaN,0,NaN,NaN,2330,8,2010,1,22,2012,NaN
75%,838,6,NaN,7856,837,1,1,NaN,0,NaN,NaN,6890,10,2013,1,37,2013,NaN
max,1115,7,NaN,41551,7388,1,1,NaN,1,NaN,NaN,75860,12,2015,1,50,2015,NaN
counts,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1014567,693861,693861,1017209,509178,509178,509178
uniques,1115,7,942,21734,4086,2,2,5,2,4,3,654,12,23,2,24,7,3


In [8]:
DataFrameSummary(df_test).summary()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
count,41088,41088,41088,NaN,41077,41088,NaN,41088,NaN,NaN,40992,25872,25872,41088,23856,23856,NaN
mean,20544.5,555.9,3.97917,NaN,0.854322,0.395833,NaN,0.443487,NaN,NaN,5088.58,7.03525,2008.64,0.580607,24.4266,2011.82,NaN
std,11861.2,320.274,2.01548,NaN,0.352787,0.489035,NaN,0.496802,NaN,NaN,7225.49,3.14302,6.8624,0.493466,14.1613,1.69217,NaN
min,1,1,1,NaN,0,0,NaN,0,NaN,NaN,20,1,1900,0,1,2009,NaN
25%,10272.8,279.75,2,NaN,1,0,NaN,0,NaN,NaN,720,4,2006,0,13,2011,NaN
50%,20544.5,553.5,4,NaN,1,0,NaN,0,NaN,NaN,2425,7,2010,1,22,2012,NaN
75%,30816.2,832.25,6,NaN,1,1,NaN,1,NaN,NaN,6480,9,2012,1,37,2013,NaN
max,41088,1115,7,NaN,1,1,NaN,1,NaN,NaN,75860,12,2015,1,49,2015,NaN
counts,41088,41088,41088,41088,41077,41088,41088,41088,41088,41088,40992,25872,25872,41088,23856,23856,23856
uniques,41088,856,7,48,2,2,2,2,4,3,554,12,23,2,21,7,3


In [9]:
print df_train.shape
print df_test.shape

(1017209, 18)
(41088, 17)


In [10]:
def create_df_merge(train_df, test_df):
    df_train = train_df.copy()
    df_test = test_df.copy()
    df_train['Id'] = df_train.index
    df_test['Sales'] = 0.
    df_test['Customers'] = 0.
    df_train['TrainTest'] = 'Train'
    df_test['TrainTest'] = 'Test'
    df_train = df_train[sorted(df_train.columns)]
    df_test = df_test[sorted(df_test.columns)]
    assert df_train.columns.tolist() == df_test.columns.tolist()
    df_merge = pd.concat([df_train, df_test])
    return df_merge

In [11]:
def create_basic_features(df):
    df.Date = pd.to_datetime(df.Date)
    df["Store"] = df["Store"].astype(str)
    df["DayOfWeek"] = df["DayOfWeek"].astype(str)
    df["Year"] = df.Date.dt.year.astype(str)
    df["Month"] = df.Date.dt.month.astype(str)
    df["Week"] = df.Date.dt.week.astype(str)
    df["Day"] = df.Date.dt.day.astype(str)
    df['StateHoliday'] = df['StateHoliday'] != '0'
    df['SchoolHoliday'] = df['SchoolHoliday'] != 0
    df['Open'] = df['Open'] != 0
    df['Promo'] = df['Promo'] != 0
    df['CompetitionDistance'] = df['CompetitionDistance'].fillna(df['CompetitionDistance'].mean())
    df.loc[df['CompetitionOpenSinceYear'] < 2000, 'CompetitionOpenSinceYear'] = 2000
    df['CompetitionOpenSinceYear'] = df['CompetitionOpenSinceYear'].fillna(2000).astype(np.int32)
    df['CompetitionOpenSinceMonth'] = df['CompetitionOpenSinceMonth'].fillna(1).astype(np.int32)
    df["CompetitionOpenSinceDateTime"] = pd.to_datetime(df.apply(lambda x: datetime.datetime(
    x.CompetitionOpenSinceYear, x.CompetitionOpenSinceMonth, 1), axis=1).astype(pd.datetime))
    df["CompetitionDaysElapsed"] = df.Date.subtract(df["CompetitionOpenSinceDateTime"]).dt.days
    competition_distance = df['CompetitionDistance'].values
    competition_elapsed = df['CompetitionDaysElapsed'].values
    df['CompetitionDistance'] = np.where(competition_elapsed>= 0., competition_distance, 0.)
    df['CompetitionDaysElapsed'] = np.where(competition_elapsed >= 0., competition_elapsed, 0.)
    df['Promo2SinceYear'] = df['Promo2SinceYear'].fillna(2009).astype(np.int32)
    df['Promo2SinceWeek'] = df['Promo2SinceWeek'].fillna(1).astype(np.int32)
    df["Promo2SinceDateTime"] = pd.to_datetime(df.apply(lambda x: Week(
    x.Promo2SinceYear, x.Promo2SinceWeek).monday(), axis=1).astype(pd.datetime))
    df["Promo2SinceDaysElapsed"] = df.Date.subtract(df["Promo2SinceDateTime"]).dt.days
    return df

In [56]:
def create_store_dictionary(merge_df):
    df = merge_df.copy()
    df = df.sort_values(['Date'])
    dict_store = {}
    for store in tqdm(np.unique(df.Store)):
        df_store = df[df.Store == store]
        sales_mean = df.loc[(df.Sales > 0.0) & (df.Store == store),"Sales"].mean()
        customer_mean = df.loc[(df.Customers > 0.0) & (df.Store == store), "Customers"].mean()
        is_open_sunday = df.loc[(df.Sales > 0.0) & (df.DayOfWeek == 7) & (df.Store == store),"Customers"].empty
        state_hols_array = df_store.StateHoliday.values
        state_hols_starting_date = df_store.Date[np.append([state_hols_array[0] == 1], (state_hols_array[:-1] == 0) & (state_hols_array[1:] == 1))].values
        school_hols_array = df_store.SchoolHoliday.values
        school_hols_starting_date = df_store.Date[np.append([school_hols_array[0] == 1], (school_hols_array[:-1] == 0) & (school_hols_array[1:] == 1))].values
        promo_array = df_store.Promo.values
        promo_starting_date = df_store.Date[np.append([promo_array[0] == 1], (promo_array[:-1] == 0) & (promo_array[1:] == 1))].values
        dict_store[store] = {
            'sales_mean' : sales_mean,
            'customer_mean' : customer_mean,
            'is_open_sunday' : is_open_sunday,
            'state_hols_array' : state_hols_starting_date,
            'school_hols_array' : school_hols_starting_date,
            'promo_array' : promo_starting_date
        }
    return dict_store

In [65]:
def get_sales_mean(dict_store, array_store):
    array_sales_mean = np.zeros(len(array_store))
    for idx in range(len(array_store)):
        array_sales_mean[idx] = dict_store[array_store[idx]]['sales_mean']
    return array_sales_mean

def get_customer_mean(dict_store, array_store):
    customer_sales_mean = np.zeros(len(array_store))
    for idx in range(len(array_store)):
        customer_sales_mean[idx] = dict_store[array_store[idx]]['customer_mean']
    return customer_sales_mean

def get_is_open_sunday(dict_store, array_store):
    is_open_sales_mean = np.zeros(len(array_store))
    for idx in range(len(array_store)):
        is_open_sales_mean[idx] = dict_store[array_store[idx]]['is_open_sunday']
    return is_open_sales_mean

def before_state_hols_date(dict_store, array_date, array_store, array_state_hols):
    result = np.zeros(len(array_store))
    for idx in range(len(array_store)):
        if array_state_hols[idx] == True:
            result[idx] = 0
        else:
            store_array_date = dict_store[array_store[idx]]['state_hols_array'].astype('datetime64[D]')
            value = store_array_date - np.datetime64(array_date[idx], 'D')
            value = value.astype(float)
            before_values = np.abs(value[np.where(value <= 0.0)])
            if len(before_values) == 0:
                result[idx] = 0
            else:
                result[idx] = np.min(before_values)
    return result

def after_state_hols_date(dict_store, array_date, array_store, array_state_hols):
    result = np.zeros(len(array_store))
    for idx in range(len(array_store)):
        if array_state_hols[idx] == True:
            result[idx] = 0
        else:
            store_array_date = dict_store[array_store[idx]]['state_hols_array'].astype('datetime64[D]')
            value = store_array_date - np.datetime64(array_date[idx], 'D')
            value = value.astype(float)
            after_values = value[np.where(value >= 0.0)]
            if len(after_values) == 0:
                result[idx] = 0
            else:
                result[idx] = np.min(after_values)
    return result

def before_school_hols_date(dict_store, array_date, array_store, array_school_hols):
    result = np.zeros(len(array_store))
    for idx in range(len(array_store)):
        if array_school_hols[idx] == True:
            result[idx] = 0
        else:
            store_array_date = dict_store[array_store[idx]]['school_hols_array'].astype('datetime64[D]')
            value = store_array_date - np.datetime64(array_date[idx], 'D')
            value = value.astype(float)
            before_values = np.abs(value[np.where(value <= 0.0)])
            if len(before_values) == 0:
                result[idx] = 0
            else:
                result[idx] = np.min(before_values)
    return result

def after_school_hols_date(dict_store, array_date, array_store, array_school_hols):
    result = np.zeros(len(array_store))
    for idx in range(len(array_store)):
        if array_school_hols[idx] == True:
            result[idx] = 0 
        else:
            store_array_date = dict_store[array_store[idx]]['school_hols_array'].astype('datetime64[D]')
            value = store_array_date - np.datetime64(array_date[idx], 'D')
            value = value.astype(float)
            after_values = value[np.where(value >= 0.0)]
            if len(after_values) == 0:
                result[idx] = 0
            else:
                result[idx] = np.min(after_values)
    return result

def before_promo_date(dict_store, array_date, array_store, array_promo):
    result = np.zeros(len(array_store))
    for idx in range(len(array_store)):
        if array_promo[idx] == True:
            result[idx] = 0
        else:
            store_array_date = dict_store[array_store[idx]]['promo_array'].astype('datetime64[D]')
            value = store_array_date - np.datetime64(array_date[idx], 'D')
            value = value.astype(float)
            before_values = np.abs(value[np.where(value <= 0.0)])
            if len(before_values) == 0:
                result[idx] = 0
            else:
                result[idx] = np.min(before_values)
    return result

def after_promo_date(dict_store, array_date, array_store, array_promo):
    result = np.zeros(len(array_store))
    for idx in range(len(array_store)):
        if array_promo[idx] == True:
            result[idx] = 0
        else:
            store_array_date = dict_store[array_store[idx]]['promo_array'].astype('datetime64[D]')
            value = store_array_date - np.datetime64(array_date[idx], 'D')
            value = value.astype(float)
            after_values = value[np.where(value >= 0.0)]
            if len(after_values) == 0:
                result[idx] = 0
            else:
                result[idx] = np.min(after_values)
    return result

In [66]:
def create_advanced_features(df, dict_store):
    array_store = df['Store'].values
    array_date = df['Date'].values
    array_state_hols = df['StateHoliday'].values
    array_school_hols = df['SchoolHoliday'].values
    array_promo = df['Promo'].values
    df['sales_mean'] = get_sales_mean(dict_store, array_store)
    df['customer_mean'] = get_customer_mean(dict_store, array_store)
    df['is_open_sunday'] = get_is_open_sunday(dict_store, array_store)
    df['prev_StateHoliday'] = before_state_hols_date(dict_store, array_date, array_store, array_state_hols)
    df['next_StateHoliday'] = after_state_hols_date(dict_store, array_date, array_store, array_state_hols)
    df['prev_SchoolHoliday'] = before_school_hols_date(dict_store, array_date, array_store, array_school_hols)
    df['next_SchoolHoliday'] = after_school_hols_date(dict_store, array_date, array_store, array_school_hols)
    df['prev_Promo'] = before_promo_date(dict_store, array_date, array_store, array_promo)
    df['next_Promo'] = after_promo_date(dict_store, array_date, array_store, array_promo)
    return df

In [96]:
def create_train_test_split(df, one_hot = True):
    all_features = ['DayOfWeek', 'Open',
                   u'Promo', u'StateHoliday', u'SchoolHoliday', u'StoreType',
       u'Assortment', u'CompetitionDistance', u'CompetitionOpenSinceMonth',
       u'CompetitionOpenSinceYear', u'Promo2', u'Promo2SinceWeek',
       u'Promo2SinceYear', u'Year', u'Month', u'Week',
       u'Day', u'CompetitionDaysElapsed',
        u'Promo2SinceDaysElapsed', u'sales_mean',
       u'customer_mean', u'is_open_sunday', u'prev_StateHoliday',
       u'next_StateHoliday', u'prev_SchoolHoliday', u'next_SchoolHoliday',
       u'prev_Promo', u'next_Promo', 'TrainTest', 'Sales']
    cat_features = ['DayOfWeek', 'StoreType', 'Assortment', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear',
                   'Promo2SinceWeek', 'Promo2SinceYear', 'Year', 'Month', 'Week', 'Day']
    for x in cat_features:
        df[x] = df[x].astype(str)
    df = df[all_features]
    if one_hot:
        df = pd.get_dummies(df, prefix = cat_features, columns= cat_features)
    else:
        df = df
    df_train = df[df['TrainTest'] == 'Train']
    df_test  = df[df['TrainTest'] == 'Test']
    
    df_X_train =  df_train.drop(['TrainTest', 'Sales'], axis = 1)
    df_X_test = df_test.drop(['TrainTest', 'Sales'], axis = 1)
    
    assert df_X_train.columns.tolist() == df_X_test.columns.tolist()
    
    X_train = df_X_train.values
    X_test = df_X_test.values
    Y_train = df_train['Sales'].values
    
    return X_train, Y_train, X_test, df_X_train.columns

In [68]:
def run_preprocessing_pipeline(df_train, df_test, one_hot = True):
    df_merge = create_df_merge(df_train, df_test)
    df_merge_basic_feat = create_basic_features(df_merge)
    dict_store = create_store_dictionary(df_merge)
    df_merge_adv_feat = create_advanced_features(df_merge, df_store)
    X_train, Y_train, X_test, column_names = create_train_test_split(df_features, one_hot)
    return  X_train, Y_train, X_test, column_names


In [17]:
df_merge = create_df_merge(df_train, df_test)

In [18]:
df_merge_basic_feat = create_basic_features(df_merge)

In [57]:
dict_store = create_store_dictionary(df_merge)

In [69]:
df_merge_adv_feat = create_advanced_features(df_merge, dict_store)

In [97]:
X_train, Y_train, X_test, column_names = create_train_test_split(df_merge_adv_feat, one_hot = True)

In [99]:
joblib.dump(df_merge_basic_feat, 'df_merge_basic_feat.pkl') 
joblib.dump(dict_store, 'dict_store.pkl') 
joblib.dump(df_merge_adv_feat, 'df_merge_adv_feat.pkl') 
joblib.dump(X_train, 'X_train.pkl') 
joblib.dump(Y_train, 'Y_train.pkl') 
joblib.dump(X_test, 'X_test.pkl')
joblib.dump(column_names, 'column_names.pkl')

['column_names.pkl']

In [3]:
df_merge_basic_feat = joblib.load('df_merge_basic_feat.pkl')
dict_store = joblib.load('dict_store.pkl')
df_merge_adv_feat = joblib.load('df_merge_adv_feat.pkl')
X_train = joblib.load('X_train.pkl')
Y_train = joblib.load('Y_train.pkl')
X_test = joblib.load('X_test.pkl')
column_names = joblib.load('column_names.pkl')

In [4]:
column_names.tolist()

['Open',
 'Promo',
 'StateHoliday',
 'SchoolHoliday',
 'CompetitionDistance',
 'Promo2',
 'CompetitionDaysElapsed',
 'Promo2SinceDaysElapsed',
 'sales_mean',
 'customer_mean',
 'is_open_sunday',
 'prev_StateHoliday',
 'next_StateHoliday',
 'prev_SchoolHoliday',
 'next_SchoolHoliday',
 'prev_Promo',
 'next_Promo',
 'DayOfWeek_1',
 'DayOfWeek_2',
 'DayOfWeek_3',
 'DayOfWeek_4',
 'DayOfWeek_5',
 'DayOfWeek_6',
 'DayOfWeek_7',
 'StoreType_a',
 'StoreType_b',
 'StoreType_c',
 'StoreType_d',
 'Assortment_a',
 'Assortment_b',
 'Assortment_c',
 'CompetitionOpenSinceMonth_1',
 'CompetitionOpenSinceMonth_10',
 'CompetitionOpenSinceMonth_11',
 'CompetitionOpenSinceMonth_12',
 'CompetitionOpenSinceMonth_2',
 'CompetitionOpenSinceMonth_3',
 'CompetitionOpenSinceMonth_4',
 'CompetitionOpenSinceMonth_5',
 'CompetitionOpenSinceMonth_6',
 'CompetitionOpenSinceMonth_7',
 'CompetitionOpenSinceMonth_8',
 'CompetitionOpenSinceMonth_9',
 'CompetitionOpenSinceYear_2000',
 'CompetitionOpenSinceYear_2001',
 'C

In [5]:
df_merge_adv_feat.head()

,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Customers,Date,DayOfWeek,Id,Open,Promo,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Sales,SchoolHoliday,StateHoliday,Store,StoreType,TrainTest,Year,Month,Week,Day,CompetitionOpenSinceDateTime,CompetitionDaysElapsed,Promo2SinceDateTime,Promo2SinceDaysElapsed,sales_mean,customer_mean,is_open_sunday,prev_StateHoliday,next_StateHoliday,prev_SchoolHoliday,next_SchoolHoliday,prev_Promo,next_Promo
0,a,1270.0,9,2008,555.0,2015-07-31,5,0,True,True,0,1,2009,NaN,5263.0,True,False,1,c,Train,2015,7,31,31,2008-09-01,2524.0,2008-12-29,2405,4759.096031,564.049936,1.0,57.0,0.0,0.0,0.0,0.0,0.0
1,a,1270.0,9,2008,546.0,2015-07-30,4,1,True,True,0,1,2009,NaN,5020.0,True,False,1,c,Train,2015,7,31,30,2008-09-01,2523.0,2008-12-29,2404,4759.096031,564.049936,1.0,56.0,0.0,0.0,0.0,0.0,0.0
2,a,1270.0,9,2008,523.0,2015-07-29,3,2,True,True,0,1,2009,NaN,4782.0,True,False,1,c,Train,2015,7,31,29,2008-09-01,2522.0,2008-12-29,2403,4759.096031,564.049936,1.0,55.0,0.0,0.0,0.0,0.0,0.0
3,a,1270.0,9,2008,560.0,2015-07-28,2,3,True,True,0,1,2009,NaN,5011.0,True,False,1,c,Train,2015,7,31,28,2008-09-01,2521.0,2008-12-29,2402,4759.096031,564.049936,1.0,54.0,0.0,0.0,0.0,0.0,0.0
4,a,1270.0,9,2008,612.0,2015-07-27,1,4,True,True,0,1,2009,NaN,6102.0,True,False,1,c,Train,2015,7,31,27,2008-09-01,2520.0,2008-12-29,2401,4759.096031,564.049936,1.0,53.0,0.0,0.0,0.0,0.0,0.0


In [6]:
DataFrameSummary(df_merge_adv_feat).summary()

,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Customers,Date,DayOfWeek,Id,Open,Promo,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Sales,SchoolHoliday,StateHoliday,Store,StoreType,TrainTest,Year,Month,Week,Day,CompetitionOpenSinceDateTime,CompetitionDaysElapsed,Promo2SinceDateTime,Promo2SinceDaysElapsed,sales_mean,customer_mean,is_open_sunday,prev_StateHoliday,next_StateHoliday,prev_SchoolHoliday,next_SchoolHoliday,prev_Promo,next_Promo
count,NaN,1.0583e+06,NaN,NaN,1.0583e+06,NaN,NaN,1.0583e+06,NaN,NaN,1.0583e+06,NaN,NaN,NaN,1.0583e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0583e+06,NaN,1.0583e+06,1.0583e+06,1.0583e+06,1.0583e+06,1.0583e+06,1.0583e+06,1.0583e+06,1.0583e+06,1.0583e+06,1.0583e+06
mean,NaN,4977.7,NaN,NaN,608.564,NaN,NaN,489655,NaN,NaN,0.503671,NaN,NaN,NaN,5549.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2902.97,NaN,1361.48,6940.16,757.689,1,30.6031,24.3145,26.1157,22.7906,5.65169,3.10491
std,NaN,7516.57,NaN,NaN,471.449,NaN,NaN,302941,NaN,NaN,0.499987,NaN,NaN,NaN,3935.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022.87,NaN,773.68,2387,355.863,0,31.0979,29.8057,23.55,23.0989,6.83561,3.32256
min,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,-881,2703.74,240.183,1,0,0,0,0,0,0
25%,NaN,450,NaN,NaN,363,NaN,NaN,223486,NaN,NaN,0,NaN,NaN,NaN,3319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,928,NaN,775,5321.14,543.676,1,4,1,6,2,0,0
50%,NaN,2060,NaN,NaN,594,NaN,NaN,488060,NaN,NaN,1,NaN,NaN,NaN,5600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2810,NaN,1576,6607.94,681.15,1,20,12,22,18,6,2
75%,NaN,6250,NaN,NaN,824,NaN,NaN,752634,NaN,NaN,1,NaN,NaN,NaN,7749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5004,NaN,1972,7967.27,867.466,1,52,38,41,37,10,6
max,NaN,75860,NaN,NaN,7388,NaN,NaN,1.01721e+06,NaN,NaN,1,NaN,NaN,NaN,41551,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5738,NaN,2453,21757.5,3403.46,1,135,195,118,194,188,16
counts,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,533034,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297
uniques,3,655,12,16,4086,990,7,1017209,2,2,2,24,7,3,21734,2,2,1115,4,2,3,12,52,31,162,5739,56,3335,1115,1114,1,136,147,119,118,21,17


In [8]:
inputs = Input(shape = (X_train.shape[1],), name='Input')
X = Dense(512, name='Dense1', activation='relu')(inputs)
X = BatchNormalization(name='BN1')(X)
X = Dropout(rate= 0.20)(X)
X = Dense(256, name='Dense2', activation='relu')(X)
X = BatchNormalization(name='BN2')(X)
X = Dropout(rate= 0.10)(X)
X = Dense(128, name='Dense3', activation='relu')(X)
X = BatchNormalization(name='BN3')(X)
X = Dropout(rate= 0.05)(X)
X = Dense(1, name='Dense4', activation='relu')(X)
NN_model = Model(inputs=inputs, outputs=X, name='NN_Model')

In [9]:
NN_model.compile('adam', 'mean_squared_error' , metrics=['accuracy'])

In [10]:
NN_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 188)               0         
_________________________________________________________________
Dense1 (Dense)               (None, 512)               96768     
_________________________________________________________________
BN1 (BatchNormalization)     (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
Dense2 (Dense)               (None, 256)               131328    
_________________________________________________________________
BN2 (BatchNormalization)     (None, 256)               1024      
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
__________

In [11]:
NN_model.fit(X_train, Y_train, epochs=1, batch_size=32)

Epoch 1/1
1017209/1017209 [==============================] - 312s 307us/step - loss: 8041733.5856 - acc: 0.0842


In [40]:
xgb_model = XGBRegressor(silent = False, n_jobs = 8)

In [ ]:
xgb_model.fit(X_train, Y_train)

In [42]:
xgb_params = {
    'learning_rate' : np.arange(0.01, 0.21, 0.01),
    'max_depth' : np.arange(3, 25, 1),
    'gamma' : np.arange(0., 3., 1.),
    "min_child_weight" : np.arange(1, 6, 1),
    'colsample_bytree' : np.arange(0.7, 1.01 ,0.1)
}

In [41]:
start = time.time()
xgb_CV_model = RandomizedSearchCV(estimator=xgb_model, param_distributions=xgb_params, n_iter= 50, scoring='neg_mean_squared_error', cv=3, verbose=4, n_jobs = 1,  return_train_score = True).fit(X_train, Y_train)
end = time.time()
end - start

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] gamma=3.0, learning_rate=0.060000000000000005, colsample_bytree=0.9999999999999999, max_depth=6, min_child_weight=1 
[CV]  gamma=3.0, learning_rate=0.060000000000000005, colsample_bytree=0.9999999999999999, max_depth=6, min_child_weight=1, score=-1058330.77212, total= 1.8min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.9min remaining:    0.0s


[CV] gamma=3.0, learning_rate=0.060000000000000005, colsample_bytree=0.9999999999999999, max_depth=6, min_child_weight=1 
[CV]  gamma=3.0, learning_rate=0.060000000000000005, colsample_bytree=0.9999999999999999, max_depth=6, min_child_weight=1, score=-1229509.33876, total= 1.8min


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.8min remaining:    0.0s


[CV] gamma=3.0, learning_rate=0.060000000000000005, colsample_bytree=0.9999999999999999, max_depth=6, min_child_weight=1 
[CV]  gamma=3.0, learning_rate=0.060000000000000005, colsample_bytree=0.9999999999999999, max_depth=6, min_child_weight=1, score=-1128630.50722, total= 1.9min


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  5.7min remaining:    0.0s


[CV] gamma=1.0, learning_rate=0.03, colsample_bytree=0.9999999999999999, max_depth=3, min_child_weight=1 
[CV]  gamma=1.0, learning_rate=0.03, colsample_bytree=0.9999999999999999, max_depth=3, min_child_weight=1, score=-1912769.86368, total= 1.0min
[CV] gamma=1.0, learning_rate=0.03, colsample_bytree=0.9999999999999999, max_depth=3, min_child_weight=1 
[CV]  gamma=1.0, learning_rate=0.03, colsample_bytree=0.9999999999999999, max_depth=3, min_child_weight=1, score=-2143057.32365, total= 1.0min
[CV] gamma=1.0, learning_rate=0.03, colsample_bytree=0.9999999999999999, max_depth=3, min_child_weight=1 
[CV]  gamma=1.0, learning_rate=0.03, colsample_bytree=0.9999999999999999, max_depth=3, min_child_weight=1, score=-2032364.68996, total= 1.0min


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  9.1min finished


692.3348560333252